In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, power_transform
from sklearn.model_selection import train_test_split
from tensorflow import keras

In [ ]:
# Load the dataset
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/My Drive/dataset/csgo_round_snapshots.csv')

# Split X and y
y = df['round_winner']
X = df.drop(['round_winner'], axis=1)
# Drop grenade info columns
X = X.drop(X.columns[X.columns.str.contains('grenade')], axis=1)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Encode categorical columns
object_cols = ['map', 'bomb_planted']
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
X_encoded = pd.DataFrame(ohe.fit_transform(X[object_cols]))
X_encoded.columns = ohe.get_feature_names_out(object_cols)
X_encoded.index = X.index

In [ ]:
# Combine with numerical features
X = pd.concat([X.drop(object_cols, axis=1), X_encoded], axis=1)

# Encode target variable
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)  # "CT" → 0, "T" → 1

# Normalize numerical features
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

In [ ]:
# Train-test split
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X_scaled, y, stratify=y, test_size=0.1, random_state=0)

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, stratify=y_train_full, test_size=0.25, random_state=0)

In [ ]:
print(X.columns.tolist())

['time_left', 'ct_score', 't_score', 'ct_health', 't_health', 'ct_armor', 't_armor', 'ct_money', 't_money', 'ct_helmets', 't_helmets', 'ct_defuse_kits', 'ct_players_alive', 't_players_alive', 'ct_weapon_ak47', 't_weapon_ak47', 'ct_weapon_aug', 't_weapon_aug', 'ct_weapon_awp', 't_weapon_awp', 'ct_weapon_bizon', 't_weapon_bizon', 'ct_weapon_cz75auto', 't_weapon_cz75auto', 'ct_weapon_elite', 't_weapon_elite', 'ct_weapon_famas', 't_weapon_famas', 'ct_weapon_g3sg1', 't_weapon_g3sg1', 'ct_weapon_galilar', 't_weapon_galilar', 'ct_weapon_glock', 't_weapon_glock', 'ct_weapon_m249', 't_weapon_m249', 'ct_weapon_m4a1s', 't_weapon_m4a1s', 'ct_weapon_m4a4', 't_weapon_m4a4', 'ct_weapon_mac10', 't_weapon_mac10', 'ct_weapon_mag7', 't_weapon_mag7', 'ct_weapon_mp5sd', 't_weapon_mp5sd', 'ct_weapon_mp7', 't_weapon_mp7', 'ct_weapon_mp9', 't_weapon_mp9', 'ct_weapon_negev', 't_weapon_negev', 'ct_weapon_nova', 't_weapon_nova', 'ct_weapon_p90', 't_weapon_p90', 'ct_weapon_r8revolver', 't_weapon_r8revolver', 'ct_

In [ ]:
# Build Neural Network model
model = keras.models.Sequential()
model.add(keras.layers.BatchNormalization(input_shape=[X_train.shape[1]]))

for _ in range(4):
    model.add(keras.layers.Dense(300, kernel_initializer="he_normal", use_bias=False))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("elu"))
    model.add(keras.layers.Dropout(rate=0.2))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/normalization/batch_normalization.py:142: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
# Output layer (softmax for multi-class classification)
model.add(keras.layers.Dense(2, activation="softmax"))  # 2 classes: CT, T

In [ ]:
# Compile model
model.compile(
    loss='sparse_categorical_crossentropy',  # Categorical loss function
    optimizer='Nadam',
    metrics=['accuracy', keras.metrics.AUC(), keras.metrics.Precision(), keras.metrics.Recall()]
)

In [ ]:
# Callbacks
reduce_lr_cb = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow import keras

# Make a train, validation and test set
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y,
 stratify=y, test_size=0.1, random_state=0)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full,
 stratify=y_train_full, test_size=0.25, random_state=0)

# Set model parameters
n_layers = 4
n_nodes = 300
regularized = False
dropout = True
epochs = 50

# Make a Keras DNN model
model = keras.models.Sequential()
model.add(keras.layers.BatchNormalization())
for n in range(n_layers):
    if regularized:
        model.add(keras.layers.Dense(n_nodes, kernel_initializer="he_normal",
         kernel_regularizer=keras.regularizers.l1(0.01), use_bias=False))
    else:
        model.add(keras.layers.Dense(n_nodes,
         kernel_initializer="he_normal", use_bias=False))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("elu"))
    if dropout:
        model.add(keras.layers.Dropout(rate=0.2))
model.add(keras.layers.Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='Nadam', metrics=['accuracy'])

# Make a callback that reduces LR on plateau
reduce_lr_cb = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                                 patience=5, min_lr=0.001)

# Make a callback for early stopping
early_stopping_cb = keras.callbacks.EarlyStopping(patience=5)

# Train DNN.
history = model.fit(np.array(X_train), np.array(y_train), epochs=epochs,
     validation_data=(np.array(X_valid), np.array(y_valid)),
      callbacks=[reduce_lr_cb, early_stopping_cb], batch_size=128)

Epoch 1/50
646/646 ━━━━━━━━━━━━━━━━━━━━ 13s 10ms/step - accuracy: 0.7204 - loss: 0.5378 - val_accuracy: 0.7522 - val_loss: 0.4684 - learning_rate: 0.0010
Epoch 2/50
646/646 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7439 - loss: 0.4768 - val_accuracy: 0.7551 - val_loss: 0.4596 - learning_rate: 0.0010
Epoch 3/50
646/646 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7496 - loss: 0.4672 - val_accuracy: 0.7581 - val_loss: 0.4591 - learning_rate: 0.0010
Epoch 4/50
646/646 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7521 - loss: 0.4631 - val_accuracy: 0.7553 - val_loss: 0.4539 - learning_rate: 0.0010
Epoch 5/50
646/646 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7501 - loss: 0.4629 - val_accuracy: 0.7587 - val_loss: 0.4504 - learning_rate: 0.0010
Epoch 6/50
646/646 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7533 - loss: 0.4596 - val_accuracy: 0.7534 - val_loss: 0.4530 - learning_rate: 0.0010
Epoch 7/50
646/646 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7566 - loss: 0.4532 

In [ ]:
# Evaluate the test set
model.evaluate(X_test, y_test)

383/383 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8033 - loss: 0.3904


[0.38804346323013306, 0.8072869777679443]

In [ ]:
# บันทึกโมเดลเป็นไฟล์ .h5
model.save("dnn_model.keras")